# Introduction

This notebook demonstrates the use of pre-trained MEGNet models to predict properties.

Author: Tsz Wai Ko (Kenko)
Email: t1ko@ucsd.edu

In [7]:
import torch
import dgl
import os
from pymatgen.core import Element, Structure, Lattice

## Import megnet related modules
from matgl.graph.converters import get_element_list, Pmg2Graph
from matgl.graph.compute import compute_pair_vector_and_distance
from matgl.models.megnet import MEGNet, MEGNetCalculator

# from matgl.utils.predictors import MEGNetCalculator
from matgl.layers.bond_expansion import BondExpansion

# MP Formation energy 

The pre-trained model is traned on the Materaisl Project mp.2018.6.1.json dataset.

In [12]:
# define the device, either "cuda" or "cpu"
device = torch.device("cpu")
# load the pre-trained MEGNet model
model = MEGNet.load()
# map the model to CPU or GPU
model = model.to(device)
# This is the structure obtained from the Materials Project.
struct = Structure.from_spacegroup("Fm-3m", Lattice.cubic(3.57743067), ["Cu"], [[0, 0, 0]])
# create a graph converter
cry_graph = Pmg2Graph(element_types=model.element_types, cutoff=model.cutoff)
# convert pymatgen structure into dgl graph
graph, graph_attrs = cry_graph.get_graph_from_structure(structure=struct)
# define the Gaussian expansion
bond_expansion = BondExpansion(rbf_type="Gaussian", initial=0.0, final=5.0, num_centers=100, width=0.5)
# compute bond vectors and distances
bond_vec, bond_dist = compute_pair_vector_and_distance(graph)
# expand the bond distance into edge attributes through Gaussian expansion
graph.edata["edge_attr"] = bond_expansion(bond_dist)
# move all necessary inputs into device
graph = graph.to(device)
graph.edata["edge_attr"] = graph.edata["edge_attr"].to(device)
graph.ndata["node_type"] = graph.ndata["node_type"].to(device)
graph_attrs = torch.tensor(graph_attrs).to(device)
# define MEGNet calculator
predictor = MEGNetCalculator(model=model, data_std=model.data_std, data_mean=model.data_mean)
Eform_pred = predictor(graph, graph_attrs)
print("The predicted formation energy for a FCC crystal is", float(Eform_pred.detach().numpy()), "eV/atom")


{'state_dict': OrderedDict([('node_embed.weight', tensor([[ 0.2391,  0.1010,  0.0555,  ..., -0.2902,  0.0376,  0.0481],
        [ 0.0210, -0.0942,  0.1350,  ..., -0.2994, -0.2566, -0.2397],
        [-0.0149, -0.0190, -0.0331,  ..., -0.0954, -0.1629,  0.0555],
        ...,
        [-0.5759,  0.8762,  0.3920,  ...,  0.8942,  0.0622,  0.6097],
        [-0.2527,  0.7182,  0.7470,  ...,  0.3337, -0.0015,  0.6665],
        [-0.2526,  0.4986,  0.2784,  ...,  0.5787, -0.0535,  0.9023]])), ('edge_encoder.layers.0.weight', tensor([[-0.0040,  0.0215,  0.1529,  ...,  0.0485,  0.0240,  0.1143],
        [-0.2633, -0.2335,  0.0182,  ..., -0.0272,  0.0284, -0.1772],
        [-0.3044, -0.2190, -0.3835,  ..., -0.0112, -0.0407,  0.0399],
        ...,
        [ 0.2111, -0.0107,  0.1740,  ..., -0.1527, -0.1654, -0.2280],
        [-1.0634, -1.0802, -0.7666,  ..., -0.7446, -0.4518, -0.5970],
        [ 1.5632,  1.2331,  1.1992,  ..., -0.2445, -0.4783, -0.5250]])), ('edge_encoder.layers.0.bias', tensor([-3.469

# MP Band gap

The pre-trained model is traned on the Materails Project mp.2018.6.1.json dataset.

In [11]:
model = MEGNet.load("MP-2019.4.1-BandGap")
# map the model to CPU or GPU
model = model.to(device)
# read structure
struct = Structure.from_spacegroup("Fm-3m", Lattice.cubic(3.57743067), ["Cu"], [[0, 0, 0]])
# create a graph converter
cry_graph = Pmg2Graph(element_types=model.element_types, cutoff=model.cutoff)
# convert pymatgen structure into dgl graph
graph, graph_attrs = cry_graph.get_graph_from_structure(structure=struct)
# define graph label ("0": PBE, "1": GLLB-SC, "2": HSE, "3": SCAN) for multi-fidelity model
graph_attrs = torch.tensor([0])
# define the Gaussian expansion
bond_expansion = BondExpansion(rbf_type="Gaussian", initial=0.0, final=6.0, num_centers=100, width=0.5)
# compute bond vectors and distances
bond_vec, bond_dist = compute_pair_vector_and_distance(graph)
# expand the bond distance into edge attributes through Gaussian expansion
graph.edata["edge_attr"] = bond_expansion(bond_dist)
# move all necessary inputs into device
graph.ndata["node_type"].to(device)
model.data_mean.to(device)
model.data_std.to(device)
graph_attrs = torch.tensor(graph_attrs).to(device)
# define MEGNet calculator
predictor = MEGNetCalculator(model=model, data_std=model.data_std, data_mean=model.data_mean)
BandGap = predictor(graph, graph_attrs)
print("The predicted PBE BandGap for a FCC Cu crystal is ", float(BandGap.detach().numpy()), "eV")

{'state_dict': OrderedDict([('node_embed.weight', tensor([[ 0.2097,  0.1751, -0.1529,  ..., -0.1116,  0.2269, -0.0190],
        [ 0.2671,  0.2182, -0.5683,  ..., -0.7295,  0.3526,  0.0642],
        [ 0.1168,  0.0504, -0.2032,  ..., -0.3724,  0.0175,  0.0108],
        ...,
        [-0.1610, -0.2858,  0.2649,  ..., -0.3606, -0.1309,  0.3828],
        [-0.2956, -0.3823,  0.2562,  ..., -0.1126, -0.3141,  0.1501],
        [-0.1923, -0.3184,  0.2706,  ..., -0.0448, -0.1976,  0.1412]])), ('attr_embed.weight', tensor([[-1.2206e-01, -7.8153e-02, -8.4857e-02,  1.3616e-01, -9.9404e-02,
          2.2256e-01,  1.4875e-01,  1.3350e-02, -3.2817e-02,  5.5671e-02,
          5.9805e-02,  1.4380e-02, -1.6309e-01,  6.1031e-02,  7.6613e-02,
          1.5576e-01],
        [ 3.3268e-01,  2.3896e-01,  1.1927e-01, -2.9558e-01,  3.8481e-01,
         -3.2706e-01, -3.5884e-01, -4.2771e-01,  4.2366e-01,  1.0296e-01,
         -2.7145e-01, -4.3383e-01,  3.3733e-01, -3.4034e-01,  3.9735e-01,
         -3.6109e-01],
  

/var/folders/w6/yrmcztx969j0r2f2v6yy3gp00000gn/T/ipykernel_26345/534914385.py:22: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  graph_attrs = torch.tensor(graph_attrs).to(device)


In [5]:
Structure.from_spacegroup?